# Split dataset and upload to huggingface
* This notebook will spilt the dataset into: training, validate and test set.
* Then upload the dataset to huggingface 

In [30]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from huggingface_hub import notebook_login

In [31]:
name = "high_vs_non_high_priority"
file_name = f"csv/clean_{name}.csv"
df = pd.read_csv(file_name)
df

,text_clean,label
0,to add link about browser support of copy comm...,0
1,improve properties of tasks definition files a...,0
2,bug exporting obj only uses one material gantt...,0
3,items with a single result in error weapon poi...,1
4,cards that are put into shadows with assault f...,0
...,...,...
430621,move resources blog post links and video from ...,0
430622,requisition line comments describe the bug req...,1
430623,have bug in create in seeds describe the bug h...,0
430624,log in notifications,0


In [32]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label
3129,NaN,0
3531,NaN,0
5819,NaN,0
6645,NaN,0
7862,NaN,0
...,...,...
416671,NaN,0
418696,NaN,0
426919,NaN,0
427334,NaN,0


In [33]:
df = df.dropna(subset=['text_clean'])
# reset index
df = df.reset_index(drop=True)
df

,text_clean,label
0,to add link about browser support of copy comm...,0
1,improve properties of tasks definition files a...,0
2,bug exporting obj only uses one material gantt...,0
3,items with a single result in error weapon poi...,1
4,cards that are put into shadows with assault f...,0
...,...,...
430282,move resources blog post links and video from ...,0
430283,requisition line comments describe the bug req...,1
430284,have bug in create in seeds describe the bug h...,0
430285,log in notifications,0


In [34]:
null_rows = df[df['text_clean'].isnull()]
null_rows

,text_clean,label


In [35]:
df.label.value_counts().to_frame()[:50]
value_counts_df = df['label'].value_counts().to_frame()
value_counts_df

,count
label,
1,215310
0,214977


In [36]:
# Split dataframe into three parts: training, validation and testing.
# Currently 80/10/10 split.
def train_validate_test_split(df, train_percent=.8, validate_percent=.1, seed=42):
    np.random.seed(seed)
    # Shuffle index of dataframe
    perm = np.random.permutation(df.index)
    
    df_length = len(df.index)
    
    # Number of row in training set
    train_end = int(train_percent * df_length)
    # Number of rows in validate set
    validate_end = int(validate_percent * df_length) + train_end
    
    # From start to train end
    train = df.iloc[perm[:train_end]]
    # From train_end to validate_end
    validate = df.iloc[perm[train_end:validate_end]]
    # From validate to the last row in dataframe.
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [37]:
# Split dataframe into three parts: training, validation and testing.
train , validate , test = train_validate_test_split(df)

In [38]:
# Convert from Pandas DataFrame to Hugging Face datasets
tds = Dataset.from_pandas(train)
vds = Dataset.from_pandas(validate)
test_ds = Dataset.from_pandas(test)

ds = DatasetDict()

ds["test"] = test_ds
ds["train"] = tds
ds["validate"] = vds

ds

DatasetDict({
    test: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 43030
    })
    train: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 344229
    })
    validate: Dataset({
        features: ['text_clean', 'label', '__index_level_0__'],
        num_rows: 43028
    })
})

In [39]:
# Uncomment this if not logged in.
#notebook_login()

In [40]:
# Push to Hugging Face Hub
ds.push_to_hub(name)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/44 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/345 [00:00<?, ?ba/s]

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/44 [00:00<?, ?ba/s]